# Examen Final: Clasificación de Mortalidad Hospitalaria  
**Machine Learning Supervisado**  

# Mateo Hernández Gualdron


*September 25, 2025*

---

## Propósito  
El propósito de este examen es evaluar el cumplimiento de los objetivos de aprendizaje planteados al comienzo del curso:

- Identificar problemas que se pueden resolver usando Machine Learning supervisado.  
- Implementar una solución de Machine Learning supervisado a problemas prácticos.  
- Evaluar el desempeño de modelos de Machine Learning supervisado.  

---

## 1. Descripción del Problema  
Predecir la mortalidad hospitalaria de pacientes críticamente enfermos es importante debido a la creciente preocupación sobre la pérdida de control de los pacientes hacia el final de la vida.  

Una predicción acertada permite tomar decisiones anticipadas para reducir la frecuencia de un proceso de muerte mecánico, doloroso y prolongado.  

El objetivo del examen es diseñar un clasificador que permita predecir la mortalidad de estos pacientes a partir de varias características fisiológicas, demográficas y de severidad de la enfermedad.  

---

## 2. Dataset  
El archivo de datos proporcionado es **`data_train.csv`**. Este archivo contiene un total de **9105 registros de pacientes**.  

Las características del dataset incluyen:  
- Información fisiológica de los pacientes.  
- Datos demográficos.  
- Información sobre la severidad de la enfermedad.  
- Indicadores de mortalidad a 2 y 6 meses.  
- Columna de mortalidad hospitalaria (`hospdead`, la etiqueta a predecir).  

---

## 3. Evaluación  
Usted es libre de emplear cualquier tipo de modelo, así como de utilizar el preprocesamiento de los datos que considere adecuado.  

La evaluación del examen final se basará en los siguientes criterios:

- **(10%)** Adecuación y preprocesamiento de los datos.  
- **(10%)** Evidencia del entrenamiento apropiado de sus modelos.  
- **(20%)** Selección del tipo de modelo, y método de selección de modelo y/o regularización utilizados, incluyendo evidencia numérica.  
- **(20%)** Calidad de su modelo final. Se debe especificar y justificar claramente el criterio de desempeño utilizado.  
  - ¿Cuál es el desempeño esperado de su modelo en datos futuros?  
  - Dé evidencia numérica.  
- **(20%)** Análisis de sus resultados.  
- **(20%)** Calidad y orden del informe.  

---

## 4. Entregables  
El entregable es un informe (**Jupyter Notebook**) del procedimiento llevado a cabo para llegar a su modelo final.  

Este informe debe estar bien estructurado e incluir la información requerida en la evaluación. Incluya **gráficas y tablas** que le permitan presentar la información de manera concisa y clara.  

El código debe estar bien estructurado y apropiadamente comentado. En el notebook deben visualizarse las ejecuciones realizadas.  


In [ ]:
#Primero: cargar el Drive:
from google.colab import drive
drive.mount('/content/Mdrive')
%cd '/content/Mdrive/MyDrive/Andes/202521/Supervisado/'
! pwd
! dir

Drive already mounted at /content/Mdrive; to attempt to forcibly remount, call drive.mount("/content/Mdrive", force_remount=True).
/content/Mdrive/MyDrive/Andes/202521/Supervisado
/content/Mdrive/MyDrive/Andes/202521/Supervisado
data_train.csv	ProyectoFinal.ipynb


# Preprocesamiento

In [ ]:
import pandas as pd
data_raw = pd.read_csv('data_train.csv',sep=',')
data_raw.drop('Unnamed: 0', axis=1, inplace=True)
data_raw.head(6)

,age,sex,dzgroup,dzclass,num.co,scoma,avtisst,race,sps,aps,...,dnr,dnrday,meanbp,hrt,resp,temp,crea,sod,adlsc,hospdead
0,62.84998,male,Lung Cancer,Cancer,0,0.0,7.000000,other,33.898438,20.0,...,no dnr,5.0,97.0,69.0,22.0,36.00000,1.199951,141.0,7.0,0
1,60.33899,female,Cirrhosis,COPD/CHF/Cirrhosis,2,44.0,29.000000,white,52.695312,74.0,...,NaN,NaN,43.0,112.0,34.0,34.59375,5.500000,132.0,1.0,1
2,52.74698,female,Cirrhosis,COPD/CHF/Cirrhosis,2,0.0,13.000000,white,20.500000,45.0,...,no dnr,17.0,70.0,88.0,28.0,37.39844,2.000000,134.0,0.0,0
3,42.38498,female,Lung Cancer,Cancer,2,0.0,7.000000,white,20.097656,19.0,...,no dnr,3.0,75.0,88.0,32.0,35.00000,0.799927,139.0,0.0,0
4,79.88495,female,ARF/MOSF w/Sepsis,ARF/MOSF,1,26.0,18.666656,white,23.500000,30.0,...,no dnr,16.0,59.0,112.0,20.0,37.89844,0.799927,143.0,2.0,0
5,93.01599,male,Coma,Coma,1,55.0,5.000000,white,19.398438,27.0,...,no dnr,4.0,110.0,101.0,44.0,38.39844,0.699951,140.0,1.0,1


## Nulos y Duplicados

In [ ]:
# nulos y duplicados
print(f'Datos nulos: {data_raw.isnull().sum().sum()}')
print(f'Datos duplicados: {data_raw.duplicated().sum().sum()}')

Datos nulos: 261
Datos duplicados: 0


In [ ]:
# Exploración de datos nulos por columna
data_raw.isnull().sum()

,0
age,0
sex,0
dzgroup,0
dzclass,0
num.co,0
scoma,1
avtisst,82
race,42
sps,1
aps,1


In [ ]:
data_raw.shape

(9105, 26)

## Transformación de columnas

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
data = data_raw.copy()

In [ ]:
categoricas = data.select_dtypes(include=['object', 'category']).columns
numericas = data.select_dtypes(exclude=['object','category']).columns
print(f'variables categoricas {categoricas}')
print(f'variables numéricas {numericas}')

variables categoricas Index(['sex', 'dzgroup', 'dzclass', 'race', 'ca', 'dnr'], dtype='object')
variables numéricas Index(['age', 'num.co', 'scoma', 'avtisst', 'sps', 'aps', 'surv2m', 'surv6m',
       'hday', 'diabetes', 'dementia', 'dnrday', 'meanbp', 'hrt', 'resp',
       'temp', 'crea', 'sod', 'adlsc', 'hospdead'],
      dtype='object')


In [ ]:
# Identificar tipos de variables
categoricas = data.select_dtypes(include=['object', 'category']).columns
numericas = data.select_dtypes(exclude=['object', 'category']).columns

print(f'Variables categóricas ({len(categoricas)}): {list(categoricas)}')
print(f'Variables numéricas ({len(numericas)}): {list(numericas)}')

# Verificar valores faltantes
print(f'\nValores faltantes por tipo:')
if len(categoricas) > 0:
    print(f'Categóricas: \n{data[categoricas].isnull().sum()}')
if len(numericas) > 0:
    print(f'Numéricas: \n{data[numericas].isnull().sum()}')

# Crear transformadores con manejo de valores faltantes
transformadores = []

# Transformador para variables categóricas (si existen)
if len(categoricas) > 0:
    pipeline_categorico = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])
    transformadores.append(('cat', pipeline_categorico, categoricas))

# Transformador para variables numéricas (si existen)
if len(numericas) > 0:
    pipeline_numerico = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', RobustScaler())
    ])
    transformadores.append(('num', pipeline_numerico, numericas))

# Crear el preprocesador solo si hay transformadores
if transformadores:
    preprocesador = ColumnTransformer(
        transformers=transformadores,
        remainder='passthrough'  # Mantener columnas no especificadas
    )

    pipeline_procesamiento = Pipeline(steps=[
        ('preprocesador', preprocesador)
    ])

    # Aplicar transformaciones
    data_proc = pipeline_procesamiento.fit_transform(data)

    # Obtener nombres de columnas transformadas
    feature_names = pipeline_procesamiento.named_steps['preprocesador'].get_feature_names_out()

    # Convertir a DataFrame
    data_proc = pd.DataFrame(data_proc, columns=feature_names, index=data.index)

    print(f'\nShape original: {data.shape}')
    print(f'Shape después del preprocesamiento: {data_proc.shape}')
    print(f'Nuevas columnas creadas: {len(feature_names) - len(data.columns)}')

    # Mostrar resultado
    data_proc.head(10)
else:
    print("No se encontraron columnas para procesar")
    data_proc = data.copy()

Variables categóricas (6): ['sex', 'dzgroup', 'dzclass', 'race', 'ca', 'dnr']
Variables numéricas (20): ['age', 'num.co', 'scoma', 'avtisst', 'sps', 'aps', 'surv2m', 'surv6m', 'hday', 'diabetes', 'dementia', 'dnrday', 'meanbp', 'hrt', 'resp', 'temp', 'crea', 'sod', 'adlsc', 'hospdead']

Valores faltantes por tipo:
Categóricas: 
sex         0
dzgroup     0
dzclass     0
race       42
ca          0
dnr        30
dtype: int64
Numéricas: 
age          0
num.co       0
scoma        1
avtisst     82
sps          1
aps          1
surv2m       1
surv6m       1
hday         0
diabetes     0
dementia     0
dnrday      30
meanbp       1
hrt          1
resp         1
temp         1
crea        67
sod          1
adlsc        0
hospdead     0
dtype: int64

Shape original: (9105, 26)
Shape después del preprocesamiento: (9105, 39)
Nuevas columnas creadas: 13


In [ ]:
data_proc.columns

Index(['cat__sex_male', 'cat__dzgroup_CHF', 'cat__dzgroup_COPD',
       'cat__dzgroup_Cirrhosis', 'cat__dzgroup_Colon Cancer',
       'cat__dzgroup_Coma', 'cat__dzgroup_Lung Cancer',
       'cat__dzgroup_MOSF w/Malig', 'cat__dzclass_COPD/CHF/Cirrhosis',
       'cat__dzclass_Cancer', 'cat__dzclass_Coma', 'cat__race_black',
       'cat__race_hispanic', 'cat__race_other', 'cat__race_white',
       'cat__ca_no', 'cat__ca_yes', 'cat__dnr_dnr before sadm',
       'cat__dnr_no dnr', 'num__age', 'num__num.co', 'num__scoma',
       'num__avtisst', 'num__sps', 'num__aps', 'num__surv2m', 'num__surv6m',
       'num__hday', 'num__diabetes', 'num__dementia', 'num__dnrday',
       'num__meanbp', 'num__hrt', 'num__resp', 'num__temp', 'num__crea',
       'num__sod', 'num__adlsc', 'num__hospdead'],
      dtype='object')

In [ ]:
data_proc['num__hospdead'].value_counts()

,count
num__hospdead,
0.0,6745
1.0,2360


# Train_Test_Split

In [ ]:
X = data_proc.drop('num__hospdead', axis=1)
y = data_proc['num__hospdead']
X

,cat__sex_male,cat__dzgroup_CHF,cat__dzgroup_COPD,cat__dzgroup_Cirrhosis,cat__dzgroup_Colon Cancer,cat__dzgroup_Coma,cat__dzgroup_Lung Cancer,cat__dzgroup_MOSF w/Malig,cat__dzclass_COPD/CHF/Cirrhosis,cat__dzclass_Cancer,...,num__diabetes,num__dementia,num__dnrday,num__meanbp,num__hrt,num__resp,num__temp,num__crea,num__sod,num__adlsc
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,-0.307692,0.454545,-0.645833,-0.2,-0.347655,0.000000,0.571429,2.000000
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.429430,-0.772727,0.250000,1.0,-1.050780,4.300049,-0.714286,0.000000
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.615385,-0.159091,-0.250000,0.4,0.351565,0.800049,-0.428571,-0.333333
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,-0.461538,-0.045455,-0.250000,0.8,-0.847655,-0.400024,0.285714,-0.333333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.538462,-0.409091,0.250000,-0.4,0.601565,-0.400024,0.857143,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.076923,0.727273,0.083333,-0.2,-0.500000,-0.100098,-0.857143,-0.333333
9101,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.538462,-0.772727,-2.083333,-1.6,0.949220,4.699463,-0.285714,-0.333333
9102,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.076923,0.772727,-0.354167,0.0,0.000000,1.499756,0.285714,0.508464
9103,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-0.307692,0.500000,0.208333,0.0,-0.148435,2.300049,-0.285714,-0.333333


In [ ]:
y

,num__hospdead
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
9100,0.0
9101,0.0
9102,0.0
9103,1.0


In [ ]:
y.value_counts().sum()

np.int64(9105)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=77)

In [ ]:
X_train.shape

(7284, 38)

In [ ]:
X_test.shape

(1821, 38)

In [ ]:
X_train

,cat__sex_male,cat__dzgroup_CHF,cat__dzgroup_COPD,cat__dzgroup_Cirrhosis,cat__dzgroup_Colon Cancer,cat__dzgroup_Coma,cat__dzgroup_Lung Cancer,cat__dzgroup_MOSF w/Malig,cat__dzclass_COPD/CHF/Cirrhosis,cat__dzclass_Cancer,...,num__diabetes,num__dementia,num__dnrday,num__meanbp,num__hrt,num__resp,num__temp,num__crea,num__sod,num__adlsc
2726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.307692,0.068182,-0.416667,-0.4,-0.300780,-0.300049,-0.428571,-0.333333
595,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.307692,1.204545,0.208333,0.6,0.152345,-0.300049,1.285714,0.346354
988,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.000000,0.136364,-0.416667,-0.2,0.800785,-0.199951,-0.285714,0.477702
298,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.307692,0.363636,-0.104167,0.2,0.300785,-0.500000,0.714286,1.666667
2282,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-0.615385,0.136364,-0.416667,-0.4,-0.550780,-0.599976,-0.285714,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,-0.384615,-0.318182,-0.666667,0.2,-0.347655,0.099854,0.285714,-0.168400
4832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.153846,1.022727,0.520833,0.0,-0.250000,0.199951,0.571429,-0.333333
7832,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.076923,-0.363636,-0.750000,0.4,-0.398435,1.300049,0.000000,-0.333333
2283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.923077,0.977273,-0.666667,-1.6,0.699220,2.099854,1.142857,-0.333333


In [ ]:
y_train

,num__hospdead
2726,0.0
595,0.0
988,0.0
298,0.0
2282,1.0
...,...
167,0.0
4832,0.0
7832,0.0
2283,0.0


# Modelo

## GridSearch

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold,GridSearchCV, StratifiedKFold
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, classification_report, auc, roc_auc_score, precision_score, f1_score, make_scorer


model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='aucpr',  # Area under Precision-Recall curve, mejor para datos desbalanceados
    random_state=42,
    enable_categorical=True    # Manejo automático de categóricas
)

param_grid = {
    # Parámetros más importantes para recall
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.15],
    'scale_pos_weight': [1, 2, 3],  # Ajusta el peso de la clase minoritaria
    'reg_alpha': [0, 0.1],  # L1 regularization
    'subsample': [0.8, 1.0]  # Submuestreo para reducir overfitting
}

recall_scorer = make_scorer(recall_score, pos_label=1)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
print(f"Total de combinaciones: {np.prod([len(v) for v in param_grid.values()])}")

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=recall_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

grid.fit(X_train, y_train)


print("Mejores parámetros:", grid.best_params_)
print(f"Mejor Recall (CV): {grid.best_score_:.4f}")

Total de combinaciones: 324
Fitting 3 folds for each of 324 candidates, totalling 972 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [19:42:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Mejores parámetros: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'reg_alpha': 0, 'scale_pos_weight': 3, 'subsample': 1.0}
Mejor Recall (CV): 0.8693


## Mejor Modelo

In [ ]:
mejor_modelo = grid.best_estimator_
y_pred = mejor_modelo.predict(X_test)
y_pred_proba = mejor_modelo.predict_proba(X_test)[:, 1]

print("\n" + "="*30)
print("MÉTRICAS EN CONJUNTO DE PRUEBA")
print("="*30)
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

print("\nReporte completo:")
print(classification_report(y_test, y_pred))


MÉTRICAS EN CONJUNTO DE PRUEBA
Recall: 0.8784
Precision: 0.7745
F1-Score: 0.8232
Accuracy: 0.8995
ROC-AUC: 0.9576

Reporte completo:
              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93      1336
         1.0       0.77      0.88      0.82       485

    accuracy                           0.90      1821
   macro avg       0.86      0.89      0.88      1821
weighted avg       0.91      0.90      0.90      1821



## Importancia de las Features

In [ ]:
feature_importance = mejor_modelo.feature_importances_
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(len(feature_importance))]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

display(importance_df.head(38))

,feature,importance
18,cat__dnr_no dnr,0.445936
22,num__avtisst,0.104456
25,num__surv2m,0.068388
30,num__dnrday,0.039700
24,num__aps,0.025524
11,cat__race_black,0.022677
21,num__scoma,0.021443
26,num__surv6m,0.019836
29,num__dementia,0.018884
28,num__diabetes,0.017045


# Conclusiones

## Acerca del modelo

Mejores parámetros: {
    'learning_rate': 0.05,     # Aprendizaje conservador
    'max_depth': 4,            # Árboles poco profundos  
    'n_estimators': 100,       # Número moderado de árboles
    'reg_alpha': 0,            # Sin regularización L1
    'scale_pos_weight': 3,     # Clave para el recall
    'subsample': 1.0           # Uso completo de datos
}


1. Balance de complejidad:

- max_depth=4: Árboles suficientemente complejos para capturar patrones médicos, pero no tan profundos como para sobreajustar

- n_estimators=100: Número "goldilocks" - ni muy pocos (underfitting) ni muchos (overfitting)

- learning_rate=0.05: Aprendizaje lento y estable que evita saltar óptimos locales

2. Desbalance de clases y su manejo:

- **scale_pos_weight=3**

Es el parámetro más crítico del XGBoost para el caso de estudio. Cada caso positivo vale 3x más que uno negativo, así que penaliza 3x los falsos positivos. Esto busca un recall más alto.

3. Overfitting

- reg_alpha=0: No necesitó regularización L1 porque los datos son suficientemente explicativos.

- subsample=1.0: Utilizó los datos sin degradación

- max_depth=4: Árboles no tan profundos, evitando overfitting


## Razones para la elección de un XGBoost en contexto médico

1. Interpretabilidad: Sin lugar a duda, una caja negra en este contexto podría no ser el mejor approach con un profesional de la salud, a quien le interesa no solamente la aplicación práctica sino también la razón científica detrás de los resultados. Esto es posible con la extracción de características junto con su importancia (DNR, APACHE scores).

2. Robustez: Maneja bien outliers, en este caso serían pacientes con condiciones extremas o anómalas / desviadas del promedio.

3. Probabilidades calibradas: Obtiene predicciones confiables para sugerir acciones con base en los resultados.

4. Feature interactions: Captura interacciones complejas entre variables médicas


**Riesgos Potenciales**

1. Sesgo de confirmación: Muy dependiente de decisiones DNR previas (Encima del 44%)

2. Generalización: Entrenado en un contexto específico, así que es imposible saber si puede generalizar incluso en otro hospital.

3. Circularidad: DNR podría haberse basado en el mismo outcome (prácticamente mirando la columna pudieron haber dicho en el 44% de las veces la clasificación binaria)


--------------------------------------------------------

El parámetro scale_pos_weight=3 fue la clave del éxito, transformando un problema de clasificación desbalanceada en una herramienta clínica viable de soporte con 87.84% de detección de casos críticos.

## Features y su explicación




El modelo conseguido con Mejores parámetros:

`{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'reg_alpha': 0, 'scale_pos_weight': 3, 'subsample': 1.0}`

Tiene un desempeño sobresaliente casi excelente para el caso clínico en cuestión:

1. Recall: 87.84% - Solo se pierden 12.16% de casos positivos. Es la métrica a optimizar, la más importante y la crítica en un problema donde se deben priorizar una sensibilidad alta ya que el costo de un falso positivo sería casi inaceptable por tratarse de la vida de una persona.

2. ROC-AUC: 95.76% - Discriminación casi perfecta entre clases

3. Accuracy: 89.95% - Clasificación correcta prácticamente en 9 de cada 10 casos

4. F1-Score: 82.32% - Balance óptimo entre precision y recall

## Interpretación de la Matriz de Confusión:

- Verdaderos Positivos: ~426 casos (88% de 485)

- Falsos Negativos: ~59 casos (12% restante) - Número bajo pero estaría a consideración del personal médico decidir si es suficiente o si se requiere todavía ser más estrictos.

- Especificidad: 91% - Clasifica correctamente a pacientes negativos

## Interpretación de Features ((buscada en navegador / LLM)):

1. Dominancia del Factor DNR (44.59%)
Interpretación Clínica:

Las decisiones de "No Resucitar" son el predictor más fuerte del outcome
Sugiere que el juicio clínico médico es altamente predictivo
El modelo captura la sabiduría clínica acumulada en estas decisiones

Implicación: El modelo está aprendiendo patrones coherentes con la práctica médica establecida.

2. Scores de Gravedad (APACHE/AVTISST) (10.45%)

Los sistemas de puntuación validados confirman su utilidad predictiva
El modelo valida herramientas clínicas existentes
Coherencia con literatura médica sobre factores pronósticos

3. Estimaciones de Supervivencia (6.84%)

Las predicciones previas de supervivencia mantienen valor predictivo
Sugiere consistencia temporal en las evaluaciones médicas


## **Aplicabilidad Práctica:**

Herramienta de Apoyo: Puede asistir en decisiones clínicas complejas
Identificación Temprana: Alto recall permite detectar casos de riesgo
Validación de Juicio Clínico: Confirma la importancia de factores ya conocidos

Limitaciones a Considerar:

Dependencia de DNR: 44.59% de la predicción se basa en decisiones médicas previas
Circularidad Potencial: ¿Las órdenes DNR se basaron en factores similares al outcome?
Generalización: ¿La población es representativa de algún lugar puntual, edad, sexo, raza, factor socioeconómico?


##**A consideración**

- Realizar análisis de subgrupos (edad, comorbilidades, etc.)

- Validación externa en otros centros médicos

- Estudio de casos donde el modelo y médicos difieren

#Final

Este modelo representa un excelente equilibrio entre rendimiento técnico y coherencia clínica, con métricas que lo hacen viable para *apoyo en decisiones médicas*, especialmente por su alto recall que minimiza casos perdidos.

No se sugeriría de ninguna manera optar por sustituir al profesional de salud.